In [21]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("baseball").getOrCreate()

In [22]:
!wget -q 'https://github.com/chadwickbureau/baseballdatabank/raw/master/core/People.csv'  # Player information
!wget -q 'https://github.com/chadwickbureau/baseballdatabank/raw/master/core/Batting.csv' # Batting records
!wget -q 'https://github.com/chadwickbureau/baseballdatabank/raw/master/core/Pitching.csv' # Pitching records
!wget -q 'https://github.com/chadwickbureau/baseballdatabank/raw/master/core/Salaries.csv'

In [23]:
df_pitcher = spark.read.csv('Pitching.csv', header=True, inferSchema=True)
df_pitcher_career = df_pitcher.groupBy('playerid').sum()
#df_pitcher_career.select(['playerid', 'sum(W)']).show()
top_ten_wins = df_pitcher_career.sort(df_pitcher_career['sum(W)'].desc()).limit(10)
#top_ten_wins.select(['playerId', 'sum(W)']).show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_wins.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+---------+
|nameFirst| nameLast|
+---------+---------+
|     Pete|Alexander|
|    Roger|  Clemens|
|      Pud|   Galvin|
|   Walter|  Johnson|
|      Tim|    Keefe|
|     Greg|   Maddux|
|  Christy|Mathewson|
|      Kid|  Nichols|
|   Warren|    Spahn|
|       Cy|    Young|
+---------+---------+



In [24]:
df_pitcher = spark.read.csv('Pitching.csv', header=True, inferSchema=True)
df_pitcher_career = df_pitcher.groupBy('playerid').sum()
#df_pitcher_career.select(['playerid', 'sum(SO)']).show()
top_ten_strikeouts = df_pitcher_career.sort(df_pitcher_career['sum(SO)'].desc()).limit(10)
#top_ten_strikeouts.select(['playerId', 'sum(SO)']).show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_strikeouts.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+--------+
|nameFirst|nameLast|
+---------+--------+
|     Bert|Blyleven|
|    Steve| Carlton|
|    Roger| Clemens|
|    Randy| Johnson|
|   Walter| Johnson|
|     Greg|  Maddux|
|  Gaylord|   Perry|
|    Nolan|    Ryan|
|      Tom|  Seaver|
|      Don|  Sutton|
+---------+--------+



In [25]:
df_pitcher = spark.read.csv('Pitching.csv', header=True, inferSchema=True)
df_pitcher_career = df_pitcher.groupBy('playerid').sum()
df_pitcher_career = df_pitcher_career.withColumn('ERA', (27 * df_pitcher_career['sum(ER)'])/df_pitcher_career['sum(IPouts)'])
#df_pitcher_career.show()
#df_pitcher_career.sort(df_pitcher_career['ERA'].desc()).show()
top_ten_era = df_pitcher_career.sort(df_pitcher_career['ERA'].desc()).limit(10)
#top_ten_era.select(['playerId', 'ERA']).show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_era.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+-----------+
|nameFirst|   nameLast|
+---------+-----------+
|      Joe|     Cleary|
|    Fritz|     Fisher|
|   Carlos|      Gomez|
|   Taylor|      Hearn|
|    Harry|   Heitmann|
|  Enrique|  Hernandez|
|    Ricky|    Pickett|
|     Andy|Sommerville|
|    Terry|  Wilshusen|
|    Frank|       Wurm|
+---------+-----------+



In [26]:
df_batting = spark.read.csv('Batting.csv', header=True, inferSchema=True)
df_batting_career = df_batting.groupBy('playerid').sum()
#df_batting_career.show()
top_ten_bs = df_batting_career.sort(df_batting_career['sum(SB)'].desc()).limit(10)
#top_ten_bs.select(['playerId', 'sum(SB)']).show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_bs.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+---------+
|nameFirst| nameLast|
+---------+---------+
|      Lou|    Brock|
|      Max|    Carey|
|       Ty|     Cobb|
|    Vince|  Coleman|
|    Eddie|  Collins|
|    Billy| Hamilton|
|   Rickey|Henderson|
|    Arlie|   Latham|
|      Tim|   Raines|
|    Honus|   Wagner|
+---------+---------+



In [27]:
df_batting = spark.read.csv('Batting.csv', header=True, inferSchema=True)
df_batting_career = df_batting.groupBy('playerid').sum()
#df_batting_career.show()
top_ten_hits = df_batting_career.sort(df_batting_career['sum(H)'].desc()).limit(10)
#top_ten_hits.select(['playerId', 'sum(H)']).show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_hits.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+-----------+
|nameFirst|   nameLast|
+---------+-----------+
|     Hank|      Aaron|
|      Cap|      Anson|
|       Ty|       Cobb|
|    Derek|      Jeter|
|     Paul|    Molitor|
|     Stan|     Musial|
|     Pete|       Rose|
|     Tris|    Speaker|
|    Honus|     Wagner|
|     Carl|Yastrzemski|
+---------+-----------+



In [28]:
df_batting = spark.read.csv('Batting.csv', header=True, inferSchema=True)
df_batting_career = df_batting.groupBy('playerid').sum()
df_batting_career = df_batting_career.withColumn('Batting-Avg', (df_batting_career['sum(H)'] + df_batting_career['sum(2B)'] + df_batting_career['sum(3B)'] + df_batting_career['sum(HR)']) / 
                                                 df_batting_career['sum(AB)'])
                                                                 
#df_batting_career.show()
top_ten_batting_avg = df_batting_career.sort(df_batting_career['Batting-Avg'].desc()).limit(10)
#top_ten_batting_avg.select(['playerId', 'Batting-Avg']).show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_batting_avg.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+-----------+
|nameFirst|   nameLast|
+---------+-----------+
|     Eric|    Cammack|
|      Don|       Eddy|
|      Roy|    Gleason|
|    Larry|   Loughlin|
|      Red|       Lutz|
|  Brandon|       Lyon|
|    Scott|Munninghoff|
|  Branden|     Pinder|
|  Charley|      Suche|
|    Allie|       Watt|
+---------+-----------+



In [29]:
df_salaries = spark.read.csv('Salaries.csv', header=True, inferSchema=True)
df_salaries_career = df_salaries.groupBy('playerid').max()
#df_salaries.show()
#df_salaries_career.show()
top_ten_salaries = df_salaries_career.sort(df_salaries_career['max(salary)'].desc()).limit(10)
#top_ten_salaries.show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_salaries.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+---------+
|nameFirst| nameLast|
+---------+---------+
|   Miguel|  Cabrera|
|   Yoenis| Céspedes|
|     Zack|  Greinke|
|    Felix|Hernandez|
|     Ryan|   Howard|
|  Clayton|  Kershaw|
|    David|    Price|
|     Alex|Rodriguez|
|   Justin|Verlander|
|   Vernon|    Wells|
+---------+---------+



In [30]:
df_salaries = spark.read.csv('Salaries.csv', header=True, inferSchema=True)
df_salaries_career = df_salaries.groupBy('playerid').avg()
#df_salaries_career.show()
top_ten_salaries = df_salaries_career.sort(df_salaries_career['avg(salary)'].desc()).limit(10)
#top_ten_salaries.show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_salaries.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+---------+
|nameFirst| nameLast|
+---------+---------+
|   Miguel|  Cabrera|
|   Yoenis| Céspedes|
|   Prince|  Fielder|
|     Ryan|   Howard|
|    Derek|    Jeter|
|      Joe|    Mauer|
|     Alex|Rodriguez|
|       CC| Sabathia|
| Masahiro|   Tanaka|
|     Mark| Teixeira|
+---------+---------+



In [31]:
df_salaries = spark.read.csv('Salaries.csv', header=True, inferSchema=True)
df_salaries_career = df_salaries.groupBy('playerid').sum()
top_ten_salaries = df_salaries_career.sort(df_salaries_career['sum(salary)'].desc()).limit(10)
#top_ten_salaries.show()
df_people = spark.read.csv('People.csv', header=True, inferSchema=True)
joined_df = top_ten_salaries.join(df_people, on=['playerID'])
joined_df.select(['nameFirst', 'nameLast']).show()

+---------+---------+
|nameFirst| nameLast|
+---------+---------+
|   Adrian|   Beltre|
|   Carlos|  Beltran|
|    Barry|    Bonds|
|   Miguel|  Cabrera|
|    Derek|    Jeter|
|   Albert|   Pujols|
|    Manny|  Ramirez|
|     Alex|Rodriguez|
|       CC| Sabathia|
|     Mark| Teixeira|
+---------+---------+

